# Working with public image data as OME-Zarr

## Setup

## What is OME-Zarr and why is it important?

Through this exercise, we'll show how a common format and access mechanism allows us to explore data from multiple public data resources in a consistent way.

## Loading and working with OME-Zarr data

In this section we'll look at:

* Loading OME-Zarr data from remote locations
* Understanding the shape of the data we have loaded
* The difference between "lazy" and full loading

In [ ]:
import dask.array as da

In [ ]:
def load_ome_zarr_from_uri(uri, path_key='0'):
    array_uri = f"{uri}/{path_key}"
    
    return da.from_zarr(array_uri)

Let's look at three differenrt examples, one from the SSBD (Systems Science of Biological Dynamics), one from the Image Data Resource (IDR), and one from the BioImage Archive (BIA). We'll see how we can use the same access mechanisms for different data repositories.

### 98-Morita-ToothEpiCellDyn (SSBD:database)

* Entry page: https://ssbd.riken.jp/database/project/98/
* Publication: https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0161336

In [ ]:
ssbd_uri = "https://ssbd.riken.jp/100118-dcacbb41/zarr/v0.4/fig2ab_trajectory_epithelialcell.zarr"

### S-BIAD634 (BioImage Archive)

* Entry page: https://www.ebi.ac.uk/biostudies/BioImages/studies/S-BIAD634
* Publication: https://www.nature.com/articles/s41597-020-00608-w

In [ ]:
bia_uri = "https://uk1s3.embassy.ebi.ac.uk/bia-integrator-data/S-BIAD634/15e5f46d-369b-49cf-9fda-15c0d0ed67ab/15e5f46d-369b-49cf-9fda-15c0d0ed67ab.zarr/0"

### idr0062 (Image Data Resource)

* Entry page: https://idr.openmicroscopy.org/webclient/?show=project-801
* Publication: https://pubmed.ncbi.nlm.nih.gov/31398189/

In [ ]:
idr_uri = "https://uk1s3.embassy.ebi.ac.uk/idr/zarr/v0.1/6001247.zarr"

In [ ]:
uris = [ssbd_uri, bia_uri, idr_uri]

In [ ]:
idr_zarr = load_ome_zarr_from_uri(uri)

### Exercise: Load all three of the OME-Zarr files

### Understanding what data we have loaded

In [ ]:
idr_zarr

In [ ]:
load_ome_zarr_from_uri(ssbd_uri)

In [ ]:
load_ome_zarr_from_uri(bia_uri)

### "Lazy" vs "full" loading

When we initially "load" the OME-Zarr image, what we're actually doing is getting the shape and location of the image arrays. We don't load all of the binary data until we need to.

In [ ]:
%%time
ssbd_array = load_ome_zarr_from_uri(ssbd_uri)

In [ ]:
%%time
ssbd_array = load_ome_zarr_from_uri(ssbd_uri)

In [ ]:
%%time
idr_array = load_ome_zarr_from_uri(idr_uri)

In [ ]:
%%time
idr_array = load_ome_zarr_from_uri(idr_uri)

In [ ]:
%%time
bia_array = load_ome_zarr_from_uri(bia_uri)

In [ ]:
%%time
bia_array = load_ome_zarr_from_uri(bia_uri)

In [ ]:
%%time
full_array = bia_array.compute()

In [ ]:
%%time
full_array = idr_array.compute()

In [ ]:
%%time
full_array = idr_array.compute()

## Visualising OME-Zarr images

Now we've looked at loading, let's examine how we can visualise these images. First we'll define two helper functions:

In [ ]:
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt

def select_single_plane(array, t, z, c):
    """Select a single plane from a lazy array, load and return it."""
    
    return array[t, c, z, :, :].compute()


def scale_to_uint8(array):
    """Given an input array, convert to uint8, including scaling to fill the
    0-255 range. 
    
    Primarily used to convert general numpy arrays into an image rendering
    suitable dtype."""

    scaled = array.astype(np.float32)

    if scaled.max() - scaled.min() == 0:
        return np.zeros(array.shape, dtype=np.uint8)

    scaled = 255 * (scaled - scaled.min()) / (scaled.max() - scaled.min())

    return scaled.astype(np.uint8)

In [ ]:
single_plane_bia = select_single_plane(bia_array, t=0, z=0, c=0)

In [ ]:
Image.fromarray(single_plane_bia)

In [ ]:
size_t, size_c, size_z, size_y, size_x = idr_array.shape

In [ ]:
z = 100

In [ ]:
single_plane_idr = select_single_plane(idr_array, t=0, z=z, c=1)

In [ ]:
Image.fromarray(scale_to_uint8(single_plane_idr))

In [ ]:
single_plane_ssbd = select_single_plane(ssbd_array, t=100, z=20, c=0)

In [ ]:
Image.fromarray(scale_to_uint8(single_plane))

In [ ]:
plt.imshow(single_plane, cmap='gray')
plt.axis('off')

In [ ]:
plt.figure()
fig, axes = plt.subplots(1, 3)
axes[0].imshow(single_plane_bia, cmap='gray')
axes[1].imshow(single_plane_idr, cmap='gray')
axes[2].imshow(single_plane_ssbd, cmap='gray')


In [ ]:
%load_ext autoreload
%autoreload 2

import sys; sys.path.append("/Users/matthewh/projects/bia-explorer/")

from bia_explorer.rendering import generate_padded_thumbnail_from_ngff_uri

In [ ]:
generate_padded_thumbnail_from_ngff_uri(ssbd_uri, dims=(512, 512))

### Making a movie

In [ ]:
!pip install --quiet microfilm

In [ ]:
from microfilm import microanim

In [ ]:
ssbd_downsampled = load_ome_zarr_from_uri(ssbd_uri, path_key='1')

In [ ]:
z = 20
selected_z_plane = ssbd_downsampled[:,:,z,:,:].compute()

In [ ]:
transposed = selected_z_plane.transpose([1, 0, 2, 3])
expanded_plane = np.expand_dims(single_plane, axis=2)

In [ ]:
anim = microanim.Microanim(data=transposed, cmaps=['pure_green', 'pure_red'], fig_scaling=5)
anim.ui

In [ ]:
import imageio
from pathlib import Path

def save_movie(anim_object, movie_name, duration=20, fps=20, quality=5, format=None):
        """Save a movie
        
        Parameters
        ----------
        movie_name: str or path object
            where to save the movie
        fps: int
            frames per second
        quality: int
            quality of images (see imageio)
        format: str
            format for export
        """

        path_obj = Path(movie_name)

        if path_obj.suffix in [".mov", ".avi", ".mpg", ".mpeg", ".mp4", ".mkv", ".wmv"]:
            writer = imageio.get_writer(
                path_obj,
                fps=fps,
                quality=quality,
                format=format,
            )
        else:
            writer = imageio.get_writer(path_obj, duration=duration, format=format)

        for t in range(anim_object.max_time):
            anim_object.update_animation(t)
            anim_object.fig.canvas.draw()
            buf = np.frombuffer(anim_object.fig.canvas.tostring_rgb(), dtype=np.uint8 )
            w,h = map(int, anim_object.fig.canvas.renderer.get_canvas_width_height())
            buf.shape = (h, w, 3)
            writer.append_data(buf)
            
        writer.close()

In [ ]:
output_fpath = "../ssbd-test.gif"
save_movie(anim, output_fpath)

In [ ]:
from IPython.display import Image as ipyImage

with open(output_fpath, "rb") as fh:
    im = ipyImage(data=fh.read())

In [ ]:
im

## Computing on OME-Zarr data

In this section, we'll look at:

* How we can run a cutting-edge segmentation algorithm (Cellpose) on OME-Zarr data
* How standardised access makes this easy to repeat across repositories
* Where reference segmentations exist, how we can compare

In [ ]:
from cellpose import models, plot

In [ ]:
model = models.Cellpose(gpu=False, model_type='cyto')

In [ ]:
cellpose_masks_idr, flows, styles, diams = model.eval(single_plane_idr, diameter=None, channels=[0, 0])

In [ ]:
cellpose_masks_bia, flows, styles, diams = model.eval(single_plane_bia, diameter=None, channels=[0, 0])

In [ ]:
fig = plt.figure(figsize=(10, 10))
plt.subplot(121)
plt.imshow(cellpose_masks_bia)

In [ ]:
cellpose_masks_ssbd, flows, styles, diams = model.eval(single_plane_ssbd, diameter=None, channels=[0, 0])

In [ ]:
fig = plt.figure(figsize=(10, 10))
plt.subplot(121)
plt.imshow(cellpose_masks)

In [ ]:
plot_array = plot.mask_overlay(single_plane_ssbd, cellpose_masks_ssbd)

In [ ]:
Image.fromarray(plot_array)

### Comparing with reference segmentation masks

For two of these images, we also have access to reference "ground truth" segmentations. Let's look at how we can load and visualise them.

In [ ]:
bia_labels_uri = "https://uk1s3.embassy.ebi.ac.uk/bia-integrator-data/S-BIAD634/906e2ace-8e99-4841-89e6-f95983632896/906e2ace-8e99-4841-89e6-f95983632896.zarr/0"

In [ ]:
bia_labels_image = load_ome_zarr_from_uri(bia_labels_uri)

In [ ]:
single_plane_bia_labels = select_single_plane(bia_labels_image, 0, 0, 0)

In [ ]:
Image.fromarray(scale_to_uint8(single_plane_bia_labels))

In [ ]:
plt.figure()
fig, axes = plt.subplots(1, 2)
axes[0].imshow(cellpose_masks_bia)
axes[1].imshow(single_plane_bia_labels)


In [ ]:
idr_labels_uri = "https://uk1s3.embassy.ebi.ac.uk/idr/zarr/v0.1/6001247.zarr/labels"

In [ ]:
idr_labels_image = load_ome_zarr_from_uri(idr_labels_uri)

In [ ]:
single_plane_idr_labels = select_single_plane(idr_labels_image, 0, z, 0)

In [ ]:
plt.figure()
fig, axes = plt.subplots(1, 2)
axes[0].imshow(cellpose_masks_idr)
axes[1].imshow(single_plane_idr_labels)


In [ ]:
"""https://kitware.github.io/itk-vtk-viewer/app/?fileToLoad=https://uk1s3.embassy.ebi.ac.uk/idr/zarr/v0.1/6001247.zarr"""

## Lazy compute